In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
import math

In [ ]:
from PIL import Image

# Original width: 600
IM_WIDTH = 300 #224

# Original height: 450
IM_HEIGHT = 225 #168

BATCH_SIZE = 16

# Random seed.
SEED = 74638

data_path = '/home/ubuntu/Data/Skin/mel_bkl/'

np.random.seed(SEED)

global_mean = np.load(data_path + 'mean_image_mel_bkl.npz')['image']
resnet_im = (Image.fromarray(global_mean.astype(np.uint8))).resize((IM_WIDTH, IM_HEIGHT))

global_mean = np.asarray(resnet_im)
print(global_mean.shape)

In [ ]:
def preprocess_image(image):
    return (image - global_mean)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import compute_class_weight

train_img = ImageDataGenerator(preprocessing_function=preprocess_image, rescale=1./255)

img_gen = ImageDataGenerator(preprocessing_function=preprocess_image, rescale=1./255)

train_gen = train_img.flow_from_directory(data_path + 'train', 
                                        target_size=(IM_HEIGHT, IM_WIDTH), 
                                        class_mode='binary', 
                                        shuffle=True, 
                                        batch_size=BATCH_SIZE, 
                                        classes=['BKL', 'MEL']
                                       )
print('Training Set Generator done.\n')
val_gen = img_gen.flow_from_directory(data_path + 'val', 
                                        target_size=(IM_HEIGHT, IM_WIDTH), 
                                        class_mode='binary', 
                                        shuffle=True, 
                                        batch_size=BATCH_SIZE, 
                                        classes=['BKL', 'MEL']
                                     )
print('Validation Set Generator done.\n')
test_gen = img_gen.flow_from_directory(data_path + 'test', 
                                        target_size=(IM_HEIGHT, IM_WIDTH), 
                                        class_mode='binary', 
                                        shuffle=False, 
                                        batch_size=BATCH_SIZE, 
                                      )
print('Test Set Generator done.\n')
y_train = train_gen.classes
weights = compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {i: weights[i] for i in range(len(weights))}

print(class_weights)
print(train_gen.class_indices)

In [ ]:
model_name = 'inceptionv3'
# model_name = 'resnet50'
num_hidden_units = 512
training_set = 'full_train'
filename = 'final_bkl_mel_' + model_name + '_' + str(num_hidden_units) + '_'+ training_set + '_1.h5'
print(filename)

In [ ]:
def f1(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return ( 2*(precision * recall) / (precision + recall + K.epsilon()) )

In [ ]:
def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    return (true_positives / (predicted_positives + K.epsilon()))

In [ ]:
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return (true_positives / (possible_positives + K.epsilon()))

In [ ]:
# from keras.models import load_model
from keras import Model
from keras.layers import Dense, GlobalAveragePooling2D, Activation, Dropout, Flatten, BatchNormalization
from keras.applications.inception_v3 import InceptionV3
import keras.regularizers

inc = InceptionV3(include_top=False, input_shape=(IM_HEIGHT, IM_WIDTH, 3), weights='imagenet')

x = inc.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(num_hidden_units)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.1)(x)
out = Dense(1, activation='sigmoid', name='new_output')(x)
model = Model(inc.input, out)

'''
Best recorded metrics:
                                 precision    recall  f1-score   support

        BKL       0.90      0.90      0.90       110
        MEL       0.90      0.90      0.90       112

avg / total       0.90      0.90      0.90       222

[[ 99  11]
 [ 11 101]]
 
AUC score: N/A

'''

last_layer_weights = model.layers[-1].get_weights()
last_layer_weights[1][0] -= 0.2

model.layers[-1].set_weights(last_layer_weights)

print(model.layers[-1].get_weights()[1])

for layer in inc.layers:
    layer.trainable = False

In [ ]:
model.summary()

In [ ]:
from keras.optimizers import Adam
model.compile(loss='binary_crossentropy', 
              optimizer=Adam(0.0001), 
              metrics=['accuracy'])

_ = model.fit_generator(train_gen, 
                        steps_per_epoch=train_gen.samples//BATCH_SIZE + 1, 
                        validation_data=val_gen, 
                        validation_steps=val_gen.samples//BATCH_SIZE + 1, 
                        shuffle=True, 
                        epochs=1)

In [ ]:
print(model.layers[-1].get_weights()[1])

In [ ]:
from keras.optimizers import Adam

for l in model.layers:
    l.trainable = True

model.compile(loss='binary_crossentropy', 
                      optimizer=Adam(lr=5e-5, decay=5e-4), 
                      metrics=['accuracy', precision, recall, f1])

print('Model compiled')

initial_epoch = 0

In [ ]:
# Recompiling model AFTER running for x epochs.
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-5, decay=5e-2), metrics=['categorical_accuracy', precision, recall, f1])
print('Model re-compiled')

print(filename)

initial_epoch = 10

In [ ]:
from keras.callbacks import LearningRateScheduler, ModelCheckpoint, EarlyStopping

# Model checkpointing.
checkpoint = ModelCheckpoint(filename, 
                             save_best_only=True, 
                             monitor='val_f1', 
                             verbose=1, mode='max')

# Preventing unnecessary training.
stopper = EarlyStopping(monitor='val_loss', mode='min', patience=3)

h = model.fit_generator(train_gen, 
                                steps_per_epoch=train_gen.samples//BATCH_SIZE + 1, 
                                validation_data=val_gen, 
                                validation_steps=val_gen.samples//BATCH_SIZE + 1, 
                                shuffle=True, 
                                epochs=12,
                                initial_epoch=0,
                                callbacks=[checkpoint, stopper], 
                                class_weight=class_weights)

fig = plt.figure()
plt.plot(h.history['loss'], 'r-')
plt.plot(h.history['val_loss'], 'b-')
plt.title('Loss')
plt.show()

fig = plt.figure()
plt.plot(h.history['f1'], 'r-')
plt.plot(h.history['val_f1'], 'b-')
plt.title('F1')
plt.show()

In [ ]:
y_pred = model.predict_generator(test_gen, verbose=1)

In [ ]:
y_true = test_gen.classes

# y_pred = np.argmax(y_pred, axis=1)
y_pred[y_pred >= 0.5] = 1
y_pred[y_pred < 0.5] = 0

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true=y_true.ravel(), 
                            y_pred=y_pred.ravel(),
                            target_names=['BKL', 'MEL']))
print(confusion_matrix(y_true=y_true.ravel(), y_pred=y_pred.ravel()))

In [ ]:
'''
             precision    recall  f1-score   support

        BKL       0.90      0.90      0.90       110
        MEL       0.90      0.90      0.90       112

avg / total       0.90      0.90      0.90       222

[[ 99  11]
 [ 11 101]]
'''